# Creando el modelo de regresión

En este apartado cargaremos los datos históricos y crearemos un modelo de regresión para hacer los cálculos de predicción. Para este ejemplo, utilizaremos el modelo SVR (`Support Vector Regression` del módulo de [scikit-learn](https://scikit-learn.org/stable/index.html) de [Python](https://www.python.org/): [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)), pero se podrían utilizar otros modelos de regresión para comparar los resultados.

Los pasos que se seguirán en este *notebook* son los siguientes:

 1. [Carga de datos históricos](#Carga-de-datos-históricos)
 2. [Preparación de los datos](#Preparación-de-los-datos)
 3. [Normalización](#Normalización)
 4. [Añadiendo información a los datos de entrada](#Añadiendo-información-a-los-datos-de-entrada)
 5. [Training/Test/Validation Set](#Training/Test/Validation-Set)
 6. [Creando el modelo](#Creando-el-modelo)
 7. [Visualizando los resultados](#Visualizando-los-resultados)

## Carga de datos históricos

Vamos a empezar cargando los datos:

In [15]:
import pandas as pd

df = pd.read_csv('C:/Users/alberto.irusta/Desktop/Deusto Generacion Historico/1 - Creacion Historico/Simulación_datos_2019-01-01_2019-12-31/TXTSimulación_datos_ 2019-01-01 _ 2019-12-31 .txt',
                 parse_dates=['ticketDate'])



df.rename(columns={"ticketDate": "Fecha", "amount": "Importe"}, inplace=True)

df = df[df['Fecha'] >= '2018-01-01']

Nuestros `dataframe` de momento sólo contiene 2 columnas:

 - `Fecha`: día y hora de la emisión del ticket (formato `%Y-%m-%d %H:%M:%S`)
 - `Importe`: importe del ticket en €

In [16]:
df.tail()

,Fecha,Importe
151863,2019-12-31 19:58:00,645.45
151864,2019-12-31 20:05:00,1575.74
151865,2019-12-31 20:11:00,7682.87
151866,2019-12-31 20:35:00,1937.30
151867,2019-12-31 20:58:00,2861.57


y el tipo de dato que contiene cada columna:
 - `Fecha`: tipo de dato `datetime`
 - `Importe`: tipo de dato `float`

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151868 entries, 0 to 151867
Data columns (total 2 columns):
Fecha      151868 non-null datetime64[ns]
Importe    151868 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.5 MB


## Preparación de los datos

Como se ha indicado al inicio, el objetivo es crear un modelo de **regresión**, por lo que tenemos que preparar los datos de forma que tengamos unos datos de entrada $X$ y otros de salida $Y$.

Para ello, vamos a intentar transformar nuestro `dataframe` para que tenga un formato similar a la siguiente tabla (en donde las ventas se van acumulando cada 15 minutos):

| Dia | 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 | 22:00 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2019-09-26 | 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 | 5 000 150 |
| 2019-09-27 | 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 | 5 500 150 |
| 2019-09-28 | 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 | 6 000 150 |

De esta manera, tendríamos los datos de entrada $X$ (en el que se introducirán más variables):

| 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 |
| --- | --- | --- | --- | --- | --- |
| 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 |
| 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 |
| 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 |

Y los datos de salida $Y$:

| 22:00 |
| --- |
| 5 000 150 |
| 5 500 150 |
| 6 000 150 |

Este es un ejemplo ilustrativo, ya que se contabilizan compras fueras del rango usual laboral (compras online u horarios especiales).

Para preparar esa tabla, vamos a empezar agregando las ventas en intervalos de 15 minutos:

In [18]:
df.index = df.pop('Fecha')
df = df.resample('15T').sum()

df.tail()

,Importe
Fecha,
2019-12-31 19:45:00,1274.68
2019-12-31 20:00:00,9258.61
2019-12-31 20:15:00,0.00
2019-12-31 20:30:00,1937.30
2019-12-31 20:45:00,2861.57


Continuaremos desglosando la `Fecha` en 2 nuevas columnas:
 - `Dia`
 - `Hora`

In [19]:
df['Dia'] = df.index.map(lambda x: x.strftime('%Y-%m-%d'))
df['Hora'] = df.index.map(lambda x: x.strftime('%H:%M'))

df.tail()

,Importe,Dia,Hora
Fecha,,,
2019-12-31 19:45:00,1274.68,2019-12-31,19:45
2019-12-31 20:00:00,9258.61,2019-12-31,20:00
2019-12-31 20:15:00,0.00,2019-12-31,20:15
2019-12-31 20:30:00,1937.30,2019-12-31,20:30
2019-12-31 20:45:00,2861.57,2019-12-31,20:45


Ahora ya estamos listos para formatear nuestro `dataframe`, para ello utilizaremos la función [crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) del módulo de [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html):

In [20]:
df = pd.crosstab(index=df['Dia'],
                 columns=[df['Hora']],
                 values=df.Importe,
                 aggfunc=sum).fillna(0).reset_index()

df.set_index('Dia', inplace=True)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49173.00,44170.43,50518.81,18013.02,21458.66,6757.40,0.00,0.0,0.0,0.0
2019-12-28,0.0,13.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90271.92,89756.44,51535.95,53132.90,40399.05,28427.47,0.00,0.0,0.0,0.0
2019-12-29,0.0,13.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3373.95,2672.59,282.55,1261.87,1356.19,352.38,80.61,0.0,0.0,0.0
2019-12-30,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,81101.25,49469.89,0.00,9214.59,2741.31,117.33,0.00,0.0,0.0,0.0
2019-12-31,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0


Para finalizar con esta parte, vamos hacer la suma acumulada de cada fila:

In [21]:
df = df.cumsum(axis=1)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5409268.18,5453438.61,5503957.42,5521970.44,5543429.10,5550186.50,5550186.50,5550186.50,5550186.50,5550186.50
2019-12-28,0.0,13.67,13.67,13.67,13.67,13.67,13.67,13.67,13.67,13.67,...,6426741.03,6516497.47,6568033.42,6621166.32,6661565.37,6689992.84,6689992.84,6689992.84,6689992.84,6689992.84
2019-12-29,0.0,13.37,13.37,13.37,13.37,13.37,13.37,13.37,13.37,13.37,...,250997.02,253669.61,253952.16,255214.03,256570.22,256922.60,257003.21,257003.21,257003.21,257003.21
2019-12-30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4809944.02,4859413.91,4859413.91,4868628.50,4871369.81,4871487.14,4871487.14,4871487.14,4871487.14,4871487.14
2019-12-31,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40,5748856.40


## Normalización

Para la normalización de los datos vamos a dividir todos los campos por el máximo valor de la tabla. De esa manera, todos nuestros valores oscilarán entre 0 y 1.

In [22]:
max_value = df['23:45'].max()
df /= max_value
df.reset_index(inplace=True)

df.tail()

Hora,Dia,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
360,2019-12-27,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.734292,0.740288,0.747146,0.749591,0.752504,0.753421,0.753421,0.753421,0.753421,0.753421
361,2019-12-28,0.0,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.872411,0.884595,0.891591,0.898804,0.904288,0.908147,0.908147,0.908147,0.908147,0.908147
362,2019-12-29,0.0,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.034072,0.034435,0.034473,0.034645,0.034829,0.034876,0.034887,0.034887,0.034887,0.034887
363,2019-12-30,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.652936,0.659651,0.659651,0.660902,0.661274,0.661290,0.661290,0.661290,0.661290,0.661290
364,2019-12-31,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.780390,0.780390,0.780390,0.780390,0.780390,0.780390,0.780390,0.780390,0.780390,0.780390


## Añadiendo información a los datos de entrada

Una vez que hemos preparado los datos para poder aplicar un modelo de regresión, vamos a añadir información adicional a los datos para intentar crear un modelo más afinado. En este caso vamos a añadir información de calendario:

 - Día de la semana
 - Días festivos
 
Esas nuevas variables van a ser variables **categóricas** por lo que crearemos [variables *dummy*](https://medium.com/hugo-ferreiras-blog/dealing-with-categorical-features-in-machine-learning-1bb70f07262d) para introducirlo en el modelo.

### Días de la semana

In [23]:
df['Dia'] = pd.to_datetime(df['Dia'])
weekdays = [
    [0, 'Lunes'],
    [1, 'Martes'],
    [2, 'Miercoles'],
    [3, 'Jueves'],
    [4, 'Viernes'],
    [5, 'Sabado'],
    [6, 'Domingo']
]
for weekday, weekday_name in weekdays:
    df[weekday_name] = df['Dia'].map(lambda x: x.weekday() == weekday)
    
df[['Dia', 'Lunes', 'Martes', 'Miercoles',
    'Jueves', 'Viernes', 'Sabado', 'Domingo']].tail()

Hora,Dia,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado,Domingo
360,2019-12-27,False,False,False,False,True,False,False
361,2019-12-28,False,False,False,False,False,True,False
362,2019-12-29,False,False,False,False,False,False,True
363,2019-12-30,True,False,False,False,False,False,False
364,2019-12-31,False,True,False,False,False,False,False


### Días Festivos

In [24]:
calendario = ['2019-01-01','2019-01-01','2019-01-06','2019-03-19',
              '2019-04-28','2019-05-15','2019-07-25','2019-08-15',
              '2019-10-12','2019-11-01','2019-12-06','2019-12-08','2019-12-25','2019-12-26']

df['Festivo'] = df['Dia'].isin(calendario)

## Training/Test/Validation Set

Por último, antes de crear el modelo vamos a dividir los datos en 3 bloques:

 - **Training Set**: 80% de los datos
 - **Test Set**: 10% de los datos
 - **Validation Set**: 10% de los datos

In [25]:
def training_test_set():
    # Training Set (80%)
    train_data = df.sample(frac=0.8, random_state=0)
    test_validation_data = df.drop(train_data.index)

    # Test/Validatin Set (10%/10%)
    test_data = test_validation_data.sample(frac=0.5, random_state=0)
    validation_data = test_validation_data.drop(test_data.index)

    # Definir la variable 'Y'
    train_y = train_data.pop('23:45')
    test_y = test_data.pop('23:45')
    validation_y = validation_data.pop('23:45')

    # Eliminamos la columna 'Dia'
    train_data.drop(['Dia'], axis=1, inplace=True)
    test_data.drop(['Dia'], axis=1, inplace=True)
    validation_data.drop(['Dia'], axis=1, inplace=True)
    
    return [train_data, test_data, validation_data, train_y, test_y, validation_y]

## Creando el modelo

Como se ha indicado en la introducción, en este ejemplo se va a utilizar el modelo [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) para la regresión y se va a elegir el `kernel` `rbf`.

No es objeto de este *notebook* explicar los detalles de este modelo, pero el lector que esté interesado en profundizar en este modelo puede ver con más detalle los algoritmos que se utilizan para hacer los cálculos en la documentación oficial de [scikit-learn](https://scikit-learn.org/stable/modules/svm.html#svm-regression).

En `SVR` hay básicamente 2 parámetros que se utilizan para ajustar el modelo:

 - `C`
 - `Epsilon`
 
a los que vamos a darle diferentes valores.

In [26]:
from sklearn.svm import SVR

def crear_modelo():
    error = -1

    for C in [0.1, 1, 100, 1000]:
        for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]:
            # Creamos el modelo con los parámetros seleccionados
            svr_rbf = SVR(kernel='rbf', C=C, gamma='auto', epsilon=epsilon)

            # Ajustamos el modelo a nuestros datos
            model = svr_rbf.fit(train_data, train_y)

            # Medir la calidad del modelo con el Test Set
            error_now = (model.predict(test_data) - test_y).std()

            # Guardar los parámetros si se ha mejorado el error
            if (error_now < error) or (error == -1):
                error = error_now
                C_good = C
                epsilon_good = epsilon
                
    return SVR(kernel='rbf', C=C_good, gamma='auto', epsilon=epsilon_good)

Como queremos crear diferentes modelos para cada hora, a la hora de crear el modelo tendremos que modificar las variables de entrada $X$, eliminando las columnas que no se van a tener en cuenta para el modelo. Para ese fin vamos a definir la siguiente función:

In [27]:
def eliminar_columnas(hour_now):
    """Eliminar las columnas que no se van a utilizar para el cálculo de las previsiones"""

    cols_drop = df.columns[(df.columns > hour_now) & (df.columns < '23:45')]
    for col in cols_drop:
        df.pop(col)

También vamos a necesitar otra función para guardar los modelos utilizando el módulo `joblib`

In [28]:
import os
import joblib


def guardar_modelo_svr(name):
    """Guardar los resultados del modelo"""

    folder = 'models/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    filename = 'all_data_model_' + name + '.sav'
    joblib.dump(model, folder + filename)

In [29]:
df_orig = df.copy()

Pongamos en marcha los cálculos...

In [30]:
for hour in range(10, 22):
    print('\nCreando modelo para las ' + str(hour) + '...')
    df = df_orig.copy()
    eliminar_columnas(str(hour) + ':00')
    train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()
    model = crear_modelo()
    model.fit(train_data, train_y)
    guardar_modelo_svr(str(hour) + '00')
    print('Modelo guardado!')
    print('Error de validación: ' + str((model.predict(validation_data) - validation_y).std()))


Creando modelo para las 10...
Modelo guardado!
Error de validación: 0.05096578614287056

Creando modelo para las 11...
Modelo guardado!
Error de validación: 0.020561691296828426

Creando modelo para las 12...
Modelo guardado!
Error de validación: 0.021044507170357454

Creando modelo para las 13...
Modelo guardado!
Error de validación: 0.01687355083110465

Creando modelo para las 14...
Modelo guardado!
Error de validación: 0.017305579861425013

Creando modelo para las 15...
Modelo guardado!
Error de validación: 0.015510280483567625

Creando modelo para las 16...
Modelo guardado!
Error de validación: 0.015504899847212343

Creando modelo para las 17...
Modelo guardado!
Error de validación: 0.014160931040416023

Creando modelo para las 18...
Modelo guardado!
Error de validación: 0.012218663202950453

Creando modelo para las 19...
Modelo guardado!
Error de validación: 0.010746137091347693

Creando modelo para las 20...
Modelo guardado!
Error de validación: 0.006466525379556625

Creando mod

## Visualizando los resultados

Cargamos las librerias de [Plotly](https://plot.ly/python/)

In [31]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

Seleccionamos los días que queremos mostrar

In [32]:
dias = df_orig.loc[df_orig.index.isin(validation_y.index), 'Dia']

Cargamos el modelo que queremos analizar:

In [33]:
hour = 10

# Cargamos el modelo
loaded_model = joblib.load('models/all_data_model_{}00.sav'.format(str(hour)))

# Creamos los datos de validación
df = df_orig.copy()
eliminar_columnas(str(hour) + ':00')
train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()

Visualizamos

In [35]:
iplot({
    'data': [go.Scatter(
                x=dias,
                y=validation_y * max_value,
                name='Real'
            ),go.Scatter(
                x=dias,
                y=loaded_model.predict(validation_data) * max_value,
                name='Predicción'
    )],
    'layout': go.Layout(
                yaxis={
                    'title': 'Ventas €'
                }
    )
})